初始已知【2】类，增量每次增【2】类，直至囊括所有类别

In [1]:
# main_process.py
# 主程序入口，进行数据加载、预处理、模型训练与测试，并评估分类准确率

import os
import pandas as pd
import numpy as np
import random
import torch
import torch.nn.functional as F
from torch.distributions import Categorical
from torch import nn
from data_loader import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from collections import Counter
from autoencoder import *
from evt import *
import argparse
import time
from datetime import timedelta
from sklearn.cluster import KMeans, DBSCAN
import warnings

start_time = time.time()

In [ ]:
# 设置随机种子，保证实验可复现
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# 解析命令行参数，支持自定义数据集名称
default_data_name = 'demo'
# parser = argparse.ArgumentParser()
# parser.add_argument('--data_name', type = str, default = default_data_name, help = 'data name')
# args = parser.parse_args()
# data_name = args.data_name
data_name = default_data_name

# 加载数据集，并进行标准化
# data_load: 特征数据, label_load: 标签
data_load, label_load = read_dataset2017(data_name)
data_load = StandardScaler().fit_transform(data_load)

# 统计每个类别的样本数，确定训练/测试划分
count_number = Counter(label_load)
min_num = np.array(list(count_number.values())).min()  # 最小类别样本数
test_per_class = 300  # 每类测试样本数
num_per_class = min_num - test_per_class  # 每类训练样本数
dim = data_load.shape[1]  # 特征维度
b_size = test_per_class   # 批量大小
loss_func = nn.MSELoss()  # 损失函数

sum_num = len(set(list((label_load))))  # 类别总数
train_num = 2  # 训练类别数（只用一个类别做已知，其余为新类）
newclass_num = sum_num - train_num  # 新类别数

# 打乱数据顺序
shun = list(range(data_load.shape[0]))
random.shuffle(shun)
data_load = data_load[shun]
label_load = label_load[shun]

# 按数字顺序排列类别索引，而不是随机排列
# allIndex = np.arange(train_num + newclass_num)
allIndex = np.random.permutation(train_num + newclass_num)


# 计算数据加载部分耗费的时间
data_loading_end = time.time()
data_loading_time = data_loading_end - start_time
print(f"=== Data Loading Time ===")
print(f"Data loading time: {str(timedelta(seconds=data_loading_time))}")

Original data size: 68193, After 1% sampling: 68193
Class distribution after oversampling:
0.1
9     4643
4     4624
1     4602
0     4588
6     4568
2     4566
14    4565
3     4563
8     4557
12    4530
5     4511
11    4504
7     4491
10    4443
13    4438
Name: count, dtype: int64
=== Data Loading Time ===
Data loading time: 0:00:34.000428


d:\Study\Github\Trident\code\data_loader.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace([np.inf, -np.inf], np.nan)


In [3]:
# 构建训练集（只包含已知类别）
data = np.zeros((num_per_class * (train_num), dim))
label = np.zeros(num_per_class * (train_num))
for pos in range(train_num):
    i = allIndex[pos]
    data[pos * num_per_class:(pos + 1) * num_per_class,:] = data_load[label_load==i][0:num_per_class, :]
    label[pos * num_per_class:(pos + 1) * num_per_class] = i


# 构建流式测试集（包含所有类别，已知类标记为原标签，未知类标记为999）
streamdata = np.zeros((test_per_class * (train_num + newclass_num), dim))
streamlabel = np.zeros(test_per_class * (train_num + newclass_num))
gtlabel = np.zeros(test_per_class * (train_num + newclass_num))
for pos in range(train_num + newclass_num):
    i = allIndex[pos]
    streamdata[pos * test_per_class:(pos + 1) * test_per_class,:] = data_load[label_load==i][-test_per_class:, :]
    gtlabel[pos * test_per_class:(pos + 1) * test_per_class] = i
    if pos < train_num:
        streamlabel[pos * test_per_class:(pos+1) * test_per_class] = i
    else:
        streamlabel[pos * test_per_class:(pos + 1) * test_per_class] = 999
        
# 输出 gtlabel 中按出现顺序的不同标签
unique_labels = pd.Series(gtlabel).unique()
print("Distinct labels in 'gtlabel':", unique_labels)


# 根据标签统计，筛选样本数大于50的类别
# 返回当前存在的类别列表
def make_lab(label):
    xianyou = pd.DataFrame(label).value_counts()
    curr_lab = []
    for j1 in xianyou.keys():
        for i in range(train_num):
            if (xianyou[j1] > 50) and (j1[0] == allIndex[i]):
                curr_lab.append(j1[0])
                break
        # if xianyou[j1] > 50:
        #     curr_lab.append(j1[0])
    return curr_lab

# 针对每个类别训练自编码器模型，并用SPOT方法确定阈值
# 返回模型列表、阈值列表、类别列表
def train(data, label, curr_lab):
    mod_ls = []
    thred_ls = []
    class_ls = []
    batch = 10
    epoch = 10
    y_in, y1, y2, y3, y4 = data_load.shape[1], 256, 128, 64, 32
    for i in curr_lab:
        class_ls.append(i)
        model = Autoencoder(y_in, y1, y2, y3, y4)
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay = 5e-4)
        # 训练自编码器
        for i2 in range(epoch):
            shun = list(range(data[label==i].shape[0]))
            random.shuffle(shun)
            for i3 in range(int(data[label==i].shape[0] / batch)):
                data_input = torch.from_numpy(data[label==i][shun][i3 * batch : (i3+1) * batch]).float()
                pred = model(data_input)
                loss = loss_func(pred, data_input)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        mod_eva = model.eval()
        mod_ls.append(model)
        # 计算重构误差，用于阈值确定
        mse_ls = []
        for i4 in range(int(data[label==i].shape[0] / batch)):
            data_input = torch.from_numpy(data[label==i][i4 * batch : (i4+1) * batch]).float()
            pred = model(data_input)
            for i5 in range(pred.shape[0]):
                loss = loss_func(pred[i5], data_input[i5])
                mse_ls.append(float(loss.detach().numpy()))
        data_input = torch.from_numpy(data[label==i][(int(data[label==i].shape[0] / batch)) * batch:]).float()
        pred = model(data_input)
        for i5 in range(pred.shape[0]):
            loss = loss_func(pred[i5], data_input[i5])
            mse_ls.append(float(loss.detach().numpy()))
        loss_list_use = np.array(mse_ls)
        q = 5e-2 # 风险参数，可调
        if len(loss_list_use) == 0:
            thred_ls.append(0)
            continue
        # if len(loss_list_use) < 50 or np.all(loss_list_use == loss_list_use[0]):
        #     thred_ls.append(np.max(loss_list_use) + 1e-3)
        #     continue
        try:
            s = SPOT(q)
            s.fit(loss_list_use, loss_list_use)
            s.initialize()
            results = s.run_simp()
            # 阈值选取
            if results['thresholds'][0] > 0:
                thred_ls.append(results['thresholds'][0])
            else:
                thred_ls.append(np.sort(s.init_data)[int(0.85 * s.init_data.size)])
        except Exception as e:
            thred_ls.append(np.max(loss_list_use) + 1e-3)
    return mod_ls, thred_ls, class_ls

# 计算数据预处理部分耗费的时间
preprocessing_end = time.time()
preprocessing_time = preprocessing_end - data_loading_end

print(f"\n=== Data Preprocessing Time ===")
print(f"Preprocessing time: {str(timedelta(seconds=preprocessing_time))}")

Distinct labels in 'gtlabel': [ 9. 11.  0. 13.  5.  8.  2.  1. 14.  4.  7. 10. 12.  3.  6.]

=== Data Preprocessing Time ===
Preprocessing time: 0:00:00.042140


In [ ]:
def ensemble_clustering(features, k=0, verbose=True):
    if features.shape[0] == 0:
        return np.array([]), np.array([])
    
    if verbose:
        print(f"Clustering {len(features)} samples...")
    
    # 1. Initialize consensus matrix
    n_samples = len(features)
    consensus_matrix = np.zeros((n_samples, n_samples))
    
    # Normalize the data for better clustering results
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(features)
    
    # 2. Perform multiple clustering with different algorithms and parameters
    warnings.filterwarnings('ignore')
    
    # 2.1 K-means with different parameters
    if verbose:
        print("Running K-means clustering...")
    for cluster_k in range(2, 12):  # Different number of clusters
        if n_samples <= cluster_k:
            if verbose:
                print(f"Skipping k={cluster_k} since sample size {n_samples} is too small")
            break
        for random_state in range(5):  # Different initializations
            kmeans = KMeans(n_clusters=cluster_k, random_state=random_state)
            labels = kmeans.fit_predict(normalized_features)
            
            # Update consensus matrix
            for i in range(n_samples):
                for j in range(i+1, n_samples):
                    if labels[i] == labels[j]:
                        consensus_matrix[i, j] += 1
                        consensus_matrix[j, i] += 1
    
    # 2.2 DBSCAN with different parameters
    if verbose:
        print("Running DBSCAN clustering...")
    for eps in [0.1, 0.2, 0.3, 0.5, 0.7, 1.0, 1.2]:
        for min_samples in [3, 5, 7, 10, 15]:
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(normalized_features)
            
            # Filter out noise points (label -1)
            valid_indices = labels != -1
            if sum(valid_indices) < 2:  # Skip if too many noise points
                continue
                
            # Update consensus matrix for non-noise points
            for i in range(n_samples):
                if labels[i] == -1:
                    continue
                for j in range(i+1, n_samples):
                    if labels[j] != -1 and labels[i] == labels[j]:
                        consensus_matrix[i, j] += 1
                        consensus_matrix[j, i] += 1
    
    # 2.3 Deep Embedded Clustering (DEC) with different parameters
    if verbose:
        print("Running DEC clustering...")
    try:
        # Define DEC autoencoder class
        class DECAutoencoder(nn.Module):
            def __init__(self, input_dim, hidden_dim, latent_dim):
                super(DECAutoencoder, self).__init__()
                self.encoder = nn.Sequential(
                    nn.Linear(input_dim, hidden_dim),
                    nn.ReLU(),
                    nn.Linear(hidden_dim, latent_dim)
                )
                self.decoder = nn.Sequential(
                    nn.Linear(latent_dim, hidden_dim),
                    nn.ReLU(),
                    nn.Linear(hidden_dim, input_dim)
                )
                
            def forward(self, x):
                z = self.encoder(x)
                x_recon = self.decoder(z)
                return x_recon, z
        
        # Function to perform DEC clustering
        def run_dec(features, n_clusters, hidden_dim, latent_dim, epochs=30):
            input_dim = features.shape[1]
            model = DECAutoencoder(input_dim, hidden_dim, latent_dim)
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
            
            # Train autoencoder
            X_tensor = torch.FloatTensor(features)
            for epoch in range(epochs):
                model.train()
                x_recon, z = model(X_tensor)
                loss = nn.MSELoss()(x_recon, X_tensor)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            # Extract latent features
            model.eval()
            _, z = model(X_tensor)
            latent_features = z.detach().numpy()
            
            # Apply KMeans on latent features
            kmeans = KMeans(n_clusters=n_clusters)
            labels = kmeans.fit_predict(latent_features)
            return labels
        
        # Run DEC with different parameters
        for n_clusters in [3, 4, 5, 6, 7, 8]:
            for hidden_dim in [64, 128]:
                for latent_dim in [32, 48, 64]:
                    try:
                        labels = run_dec(normalized_features, n_clusters, hidden_dim, latent_dim)
                        
                        # Update consensus matrix
                        for i in range(n_samples):
                            for j in range(i+1, n_samples):
                                if labels[i] == labels[j]:
                                    consensus_matrix[i, j] += 1
                                    consensus_matrix[j, i] += 1
                    except Exception as e:
                        if verbose:
                            print(f"DEC failed with parameters: {n_clusters}, {hidden_dim}, {latent_dim}. Error: {e}")
    except Exception as e:
        if verbose:
            print(f"Skipping DEC due to error: {e}")
    
    # 3. If k is not specified, find optimal number of clusters using silhouette score
    if k <= 0:
        if verbose:
            print("\nFinding optimal number of clusters...")
        max_silhouette = -1
        best_k = 2  # default
        silhouette_scores = []
        
        for test_k in range(2, min(41, n_samples)):
            kmeans = KMeans(n_clusters=test_k, random_state=42)
            labels = kmeans.fit_predict(consensus_matrix)
            
            try:
                silhouette = silhouette_score(consensus_matrix, labels)
                silhouette_scores.append(silhouette)
                if verbose:
                    print(f"K={test_k}, Silhouette Score={silhouette:.4f}")
                
                if silhouette > max_silhouette:
                    max_silhouette = silhouette
                    best_k = test_k
            except Exception as e:
                if verbose:
                    print(f"Could not compute silhouette score for k={test_k}")
        
        if verbose:
            print(f"\nOptimal number of clusters: {best_k}")
        final_k = best_k
    else:
        final_k = k
        if verbose:
            print(f"\nUsing specified number of clusters: {final_k}")
    
    # 4. Perform final clustering with optimal/specified k
    final_kmeans = KMeans(n_clusters=final_k, random_state=42)
    predicted_clusters = final_kmeans.fit_predict(consensus_matrix)
    
    return predicted_clusters, consensus_matrix

In [5]:
# 主流程入口
if __name__ == '__main__':
    print('=== Initializing ===')
    # 获取当前类别
    curr_num = train_num
    curr_lab = allIndex[:curr_num]
    # 训练初始模型
    mod_ls, thred_ls, class_ls = train(data, label, curr_lab)
    
    res_ls = []  # 预测结果列表
    # 先预测已知的train_num个类别
    for i5 in range(test_per_class * train_num):
        # 对当前样本用所有模型计算重构误差
        data_input = torch.from_numpy(streamdata[i5]).float()
        mse_test = []
        for model in mod_ls:
            mod_eva = model.eval()
            pred = model(data_input)
            loss = loss_func(pred, data_input)
            mse_test.append(float(loss.detach().numpy()))
        # 判断是否为新类
        cand_res = np.array(mse_test)[np.array(mse_test) < np.array(thred_ls)]
        if len(cand_res) == 0:
            res_ls.append(999)
        else:
            min_loss_res = cand_res.min()
            res_ls.append(class_ls[mse_test.index(min_loss_res)])
    updatedata = data
    updatelabel = label
    # 开始处理增量数据
    for i in range(int((sum_num - train_num + 1) // 2)):
        curr_num += 2
        if curr_num <= sum_num:
            print(f"=== Incremental Progress: {curr_num}/{sum_num} classes ===")
            # 正常增量2类
            curr_lab = allIndex[:curr_num]
            print('Current labels:', curr_lab)
            unknown_samples = []
            unknown_indices = []
            real_label = []
            for i5 in range(test_per_class * (curr_num - 2), test_per_class * curr_num):
                # 对当前样本用所有模型计算重构误差
                data_input = torch.from_numpy(streamdata[i5]).float()
                mse_test = []
                for model in mod_ls:
                    mod_eva = model.eval()
                    pred = model(data_input)
                    loss = loss_func(pred, data_input)
                    mse_test.append(float(loss.detach().numpy()))
                # 判断是否为新类
                cand_res = np.array(mse_test)[np.array(mse_test) < np.array(thred_ls)]
                if len(cand_res) == 0:
                    res_ls.append(999)
                    # Record the unknown sample and its index for later clustering
                    unknown_samples.append(streamdata[i5])
                    unknown_indices.append(i5)  # Index of the newly added 999 in res_ls
                    # The streamdata index is i5, so we can get the true label from gtlabel
                    real_label.append(gtlabel[i5])  # Get the true label for this unknown sample
                else:
                    min_loss_res = cand_res.min()
                    res_ls.append(class_ls[mse_test.index(min_loss_res)])
            # Check if we have enough unknown samples to cluster
            if len(unknown_samples) > 1:
                print(f"Found {len(unknown_samples)} unknown samples for clustering")
                print("Starting ensemble clustering to identify new classes...")
                
                # Convert the list of unknown samples to a numpy array
                unknown_samples_array = np.array(unknown_samples)
                
                # Apply ensemble clustering to identify the two new classes
                cluster_labels, _ = ensemble_clustering(unknown_samples_array, k=2, verbose=False)
                
                # Update the results list with the newly assigned cluster IDs
                # Map cluster 0,1 to the next class IDs (which should be curr_num-2 and curr_num-1)
                # Check which real labels are in each cluster and map accordingly
                cluster_0_samples = [gtlabel[unknown_indices[i]] for i in range(len(unknown_indices)) if cluster_labels[i] == 0]

                # Count occurrences of each label in cluster 0
                cluster_0_counts = {}
                for label in cluster_0_samples:
                    cluster_0_counts[label] = cluster_0_counts.get(label, 0) + 1

                # Determine which label is more common in cluster 0
                if len(cluster_0_counts) > 0:
                    most_common_label_in_cluster_0 = max(cluster_0_counts, key=cluster_0_counts.get)
                    
                    # If the most common label in cluster 0 is the first new class (curr_lab[-2]),
                    # maintain original mapping, otherwise swap
                    if most_common_label_in_cluster_0 == curr_lab[-2]:
                        new_class_ids = [curr_lab[-2], curr_lab[-1]]
                    else:
                        new_class_ids = [curr_lab[-1], curr_lab[-2]]
                else:
                    new_class_ids = [curr_lab[-2], curr_lab[-1]]
                
                for i, idx in enumerate(unknown_indices):
                    # Map the cluster label to the appropriate new class ID
                    new_class_id = new_class_ids[cluster_labels[i]]
                    res_ls[idx] = new_class_id
                
                # Calculate AMI for the ensemble clustering
                ensemble_ami = adjusted_mutual_info_score(real_label, cluster_labels)
                print(f"Ensemble Clustering AMI: {ensemble_ami:.4f}")

                
                # Train new models for these newly discovered classes
                # Collect the newly discovered class samples for training
                # Only add the samples we just clustered and labeled
                new_class_data = []
                new_class_labels = []
                
                for i, idx in enumerate(unknown_indices):
                    new_class_data.append(streamdata[idx])
                    new_class_labels.append(new_class_ids[cluster_labels[i]])
                
                # Convert to numpy arrays for concatenation
                new_class_data = np.array(new_class_data)
                new_class_labels = np.array(new_class_labels)
                
                # Update the training data with new class samples
                updatedata = np.concatenate([updatedata, new_class_data], axis=0)
                updatelabel = np.concatenate([updatelabel, new_class_labels], axis=0)
                mod_ls, thred_ls, class_ls = train(updatedata, updatelabel, curr_lab)
                print('*** Update model with new classes ***')
            else:
                print("No unknown samples found for clustering")
            # Calculate current prediction accuracy
            # Get current predictions and true labels up to this point
            current_preds = np.array(res_ls)
            current_true = gtlabel[:len(res_ls)]
            current_accuracy = accuracy_score(current_true, current_preds)

            # Calculate F1 score and recall - using macro averaging to handle multiple classes
            current_f1 = f1_score(current_true, current_preds, average='macro', zero_division=0)
            current_recall = recall_score(current_true, current_preds, average='macro', zero_division=0)

            print(f"Current Prediction Metrics:")
            print(f"  Accuracy: {current_accuracy:.4f}")
            print(f"  F1 Score (macro): {current_f1:.4f}")
            print(f"  Recall (macro): {current_recall:.4f}")
        else:
            # 最后一次增量，可能只增加1类
            curr_num = sum_num
            print(f"=== Incremental Progress: {curr_num}/{sum_num} classes ===")
            curr_lab = allIndex[:curr_num]
            print('Current labels:', curr_lab)
            for i5 in range(test_per_class * (curr_num - 1), test_per_class * curr_num):
                # 对当前样本用所有模型计算重构误差
                data_input = torch.from_numpy(streamdata[i5]).float()
                mse_test = []
                for model in mod_ls:
                    mod_eva = model.eval()
                    pred = model(data_input)
                    loss = loss_func(pred, data_input)
                    mse_test.append(float(loss.detach().numpy()))
                # 判断是否为新类
                cand_res = np.array(mse_test)[np.array(mse_test) < np.array(thred_ls)]
                if len(cand_res) == 0:
                    res_ls.append(curr_lab[-1])  # 最后一类
                else:
                    min_loss_res = cand_res.min()
                    res_ls.append(class_ls[mse_test.index(min_loss_res)])
        
    print('=== Incremental Process Completed ===')
    # 计算最终分类性能指标
    print('\n=== Final Classification Results ===')
    # 将所有预测结果和真实标签转换为NumPy数组
    all_predictions = np.array(res_ls)
    all_true_labels = gtlabel[:len(res_ls)]

    # 计算整体准确率
    accuracy = accuracy_score(all_true_labels, all_predictions)
    print(f'Overall Accuracy: {accuracy:.4f}')

    # 计算F1分数 (宏平均)
    f1 = f1_score(all_true_labels, all_predictions, average='macro', zero_division=0)
    print(f'F1 Score (macro): {f1:.4f}')

    # 计算召回率 (宏平均)
    recall = recall_score(all_true_labels, all_predictions, average='macro', zero_division=0)
    print(f'Recall (macro): {recall:.4f}')

    # 计算精确率 (宏平均)
    precision = precision_score(all_true_labels, all_predictions, average='macro', zero_division=0)
    print(f'Precision (macro): {precision:.4f}')

    # 计算每个类别的指标
    print('\n=== Per-Class Metrics ===')
    # 获取唯一类别
    unique_classes = np.unique(np.concatenate((all_true_labels, all_predictions)))
    unique_classes = sorted(unique_classes[unique_classes != 999])  # 排除999标签

    class_report = classification_report(all_true_labels, all_predictions, 
                                        labels=unique_classes,
                                        zero_division=0,
                                        output_dict=True)

    # 打印每个类别的性能指标
    for class_label in unique_classes:
        print(f'Class {class_label}:')
        print(f'  Precision: {class_report[str(class_label)]["precision"]:.4f}')
        print(f'  Recall: {class_report[str(class_label)]["recall"]:.4f}')
        print(f'  F1-score: {class_report[str(class_label)]["f1-score"]:.4f}')
        print(f'  Support: {class_report[str(class_label)]["support"]}')

    # Calculate clustering time and total execution time
    clustering_end = time.time()
    clustering_time = clustering_end - preprocessing_end

    print(f"\n=== Clustering Time ===")
    print(f"Clustering time: {str(timedelta(seconds=clustering_time))}")


    print(f"\n=== Total Time ===")
    total_time = clustering_end - start_time
    print(f"Total execution time: {str(timedelta(seconds=total_time))}")

=== Initializing ===
=== Incremental Progress: 4/15 classes ===
Current labels: [ 9 11  0 13]
Found 532 unknown samples for clustering
Starting ensemble clustering to identify new classes...
Ensemble Clustering AMI: 0.2001
*** Update model with new classes ***
Current Prediction Metrics:
  Accuracy: 0.7967
  F1 Score (macro): 0.6464
  Recall (macro): 0.6373
=== Incremental Progress: 6/15 classes ===
Current labels: [ 9 11  0 13  5  8]
Found 592 unknown samples for clustering
Starting ensemble clustering to identify new classes...
Ensemble Clustering AMI: 1.0000
*** Update model with new classes ***
Current Prediction Metrics:
  Accuracy: 0.8600
  F1 Score (macro): 0.7441
  Recall (macro): 0.7371
=== Incremental Progress: 8/15 classes ===
Current labels: [ 9 11  0 13  5  8  2  1]
Found 244 unknown samples for clustering
Starting ensemble clustering to identify new classes...
Ensemble Clustering AMI: 0.1802
*** Update model with new classes ***
Current Prediction Metrics:
  Accuracy: 0.7